In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/ERA_V2/Assignment13/gradio

/content/drive/MyDrive/ERA_V2/Assignment13/gradio


In [3]:
!pip install grad-cam
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 23.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  U

In [4]:
import torch, torchvision
from torchvision import transforms
import numpy as np
import gradio as gr
from PIL import Image
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from resnet import ResNet18
import gradio as gr


In [5]:
model = ResNet18()


In [6]:
model.load_state_dict(torch.load("/content/drive/MyDrive/ERA_V2/Assignment13/gradio/model.pth", map_location=torch.device('cpu')), strict=False)

<All keys matched successfully>

In [7]:
import torch
import torchvision
import torchvision.transforms as transforms

def get_cifar10_dataloaders(batch_size=64):

    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

    return testloader

testloader = get_cifar10_dataloaders()

100%|██████████| 170498071/170498071 [00:03<00:00, 45303615.45it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


In [31]:
def get_misclassified_data(model, device, test_loader):
    """
    Function to run the model on test set and return misclassified images
    :param model: Network Architecture
    :param device: CPU/GPU
    :param test_loader: DataLoader for test set
    """
    # Prepare the model for evaluation i.e. drop the dropout layer
    model.eval()

    # List to store misclassified Images
    misclassified_data = []

    # Reset the gradients
    with torch.no_grad():
        # Extract images, labels in a batch
        for data, target in test_loader:

            # Migrate the data to the device
            data, target = data.to(device), target.to(device)

            # Extract single image, label from the batch
            for image, label in zip(data, target):

                # Add batch dimension to the image
                image = image.unsqueeze(0)

                # Get the model prediction on the image
                output = model(image)

                # Convert the output from one-hot encoding to a value
                pred = output.argmax(dim=1, keepdim=True)

                # If prediction is incorrect, append the data
                if pred != label:
                    misclassified_data.append((image, label, pred))
    return misclassified_data
##################
misclassified_data = get_misclassified_data(model,'cpu', testloader)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [37]:
inv_normalize = transforms.Normalize(
    mean=[-0.50/0.23, -0.50/0.23, -0.50/0.23],
    std=[1/0.23, 1/0.23, 1/0.23]
)
classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

def resize_image_pil(image, new_width, new_height):

    # Convert to PIL image
    img = Image.fromarray(np.array(image))

    # Get original size
    width, height = img.size

    # Calculate scale
    width_scale = new_width / width
    height_scale = new_height / height
    scale = min(width_scale, height_scale)

    # Resize
    resized = img.resize((int(width*scale), int(height*scale)), Image.NEAREST)

    # Crop to exact size
    resized = resized.crop((0, 0, new_width, new_height))

    return resized

def inference(input_img, is_grad_cam=True, transparency = 0.5,layer='layer2', target_layer_number = -1, top_predictions=3):
    input_img = resize_image_pil(input_img, 32, 32)

    input_img = np.array(input_img)
    org_img = input_img
    input_img = input_img.reshape((32, 32, 3))
    transform = transforms.ToTensor()
    input_img = transform(input_img)
    input_img = input_img
    input_img = input_img.unsqueeze(0)
    outputs = model(input_img)
    softmax = torch.nn.Softmax(dim=0)
    o = softmax(outputs.flatten())
    confidences = {classes[i]: float(o[i]) for i in range(10)}
    _, prediction = torch.max(outputs, 1)
    if is_grad_cam:
        if layer == 'layer1':
          target_layers = [model.layer1[target_layer_number]]
          cam = GradCAM(model=model, target_layers=target_layers)
          grayscale_cam = cam(input_tensor=input_img, targets=None)
          grayscale_cam = grayscale_cam[0, :]
          visualization = show_cam_on_image(org_img/255, grayscale_cam, use_rgb=True, image_weight=transparency)

        if layer == 'layer2':
          target_layers = [model.layer2[target_layer_number]]
          cam = GradCAM(model=model, target_layers=target_layers)
          grayscale_cam = cam(input_tensor=input_img, targets=None)
          grayscale_cam = grayscale_cam[0, :]
          visualization = show_cam_on_image(org_img/255, grayscale_cam, use_rgb=True, image_weight=transparency)

        if layer == 'layer3':
          target_layers = [model.layer3[target_layer_number]]
          cam = GradCAM(model=model, target_layers=target_layers)
          grayscale_cam = cam(input_tensor=input_img, targets=None)
          grayscale_cam = grayscale_cam[0, :]
          visualization = show_cam_on_image(org_img/255, grayscale_cam, use_rgb=True, image_weight=transparency)

        if layer == 'layer4':
          target_layers = [model.layer4[target_layer_number]]
          cam = GradCAM(model=model, target_layers=target_layers)
          grayscale_cam = cam(input_tensor=input_img, targets=None)
          grayscale_cam = grayscale_cam[0, :]
          visualization = show_cam_on_image(org_img/255, grayscale_cam, use_rgb=True, image_weight=transparency)
    else:
        visualization = None

    # Sort the confidences dictionary based on confidence values
    sorted_confidences = dict(sorted(confidences.items(), key=lambda item: item[1], reverse=True))

    # Pick the top n predictions
    top_n_confidences = dict(list(sorted_confidences.items())[:top_predictions])

    return classes[prediction[0].item()], visualization, top_n_confidences

title = "CIFAR10 trained on ResNet18 Model with GradCAM"
description = "A simple Gradio interface to infer on ResNet model, and get GradCAM results"

def update(name):
    return f"Welcome to Gradio, {name}!"

demo1 = gr.Interface(
    inference,
    inputs = [
        gr.Image(width=256, height=256, label="Input Image"),
        gr.Checkbox(label="Show GradCAM"),
        gr.Slider(0, 1, value = 0.5, label="Overall Opacity of Image"),
        gr.Dropdown(["layer1", "layer2", "layer3", "layer4"], label="Layer"),
        gr.Slider(-2, -1, value = -2, step=1, label="Which Layer?"),
        gr.Slider(2, 10, value=3, step=1, label="Number of Top Classes")
        ],
    outputs = [
        "text",
        gr.Image(width=256, height=256, label="Output"),
        gr.Label(label="Top Classes")
        ],
    title = title,
    description = description,
)




def plot_misclassified_images(num_images):
    images = []
    true_labels = []
    predicted_labels = []

    for i, (image, true_label, predicted_label) in enumerate(misclassified_data[:num_images]):
        image = image.squeeze(0).permute(1, 2, 0).cpu().numpy()  # Remove batch dimension and convert to numpy
        true_label = classes[true_label.item()]  # Convert true label index to class name
        predicted_label = classes[predicted_label.item()]  # Convert predicted label index to class name

    return images, true_labels, predicted_labels


demo2=gr.Interface(
    plot_misclassified_images,
    inputs = [
        gr.Number(label="Images to display"),
        ],
    outputs = [
        gr.Gallery(label="Misclassified Images", show_label=True, elem_id="gallery",  object_fit="contain", height="auto"),
        gr.Label(label="True Labels"),
        gr.Label(label="Predicted Labels"),
    ],
    title = "MissClassified Images",
    description = "Enter the number of image to be displayed"
)



demo = gr.TabbedInterface([demo1, demo2], ["Output", "MissClassified Images"])
demo.launch()




Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://f07fb41335fd5369c3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
